In [270]:
import pandas as pd
import numpy as np
from collections import Counter  # Make sure this import is present
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, average_precision_score

In [271]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Telco Preprocess


In [272]:
def preprocess_dataset_telco(filepath, target_col):
    """
    Generic preprocessing function for a dataset.
    - Handles missing values, duplicates, encoding, and scaling.

    Parameters:
    - filepath: Path to the CSV file.
    - target_col: Name of the target column.

    Returns:
    - X_train, X_test, y_train, y_test: Processed dataset split into training and testing sets.
    """
    # Load the dataset
    df = pd.read_csv(filepath)

    print(f"Initial shape: {df.shape}")

    # Convert 'TotalCharges' (or any other numerical column with spaces) to NaN, then to numeric
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

    # Report missing values and duplicates
    missing_count = df.isna().sum().sum()
    duplicate_count = df.duplicated().sum()
    print(f"Missing values: {missing_count}")
    print(f"Duplicated values: {duplicate_count}")

    # Drop rows with missing target values (Churn)
    df = df.dropna(subset=[target_col])

    # Drop duplicated rows
    df = df.drop_duplicates()

    # Fill numeric missing values with the mean of respective columns
    num_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df[num_cols] = df[num_cols].fillna(df[num_cols].mean())

    # Fill categorical missing values with the mode of respective columns
    cat_cols = df.select_dtypes(include=['object']).columns
    df[cat_cols] = df[cat_cols].fillna(df[cat_cols].mode().iloc[0])

    # Drop irrelevant columns (like 'customerID')
    if 'customerID' in df.columns:
        df = df.drop(columns=['customerID'])

    # Identify binary and nominal columns
    binary_cols = df.columns[df.nunique() == 2].tolist()
    nominal_cols = df.select_dtypes(include=['object']).columns.difference(binary_cols).tolist()

    print(f"Binary columns: {binary_cols}")
    print(f"Nominal columns: {nominal_cols}")

    # Encode binary columns using LabelEncoder
    for col in binary_cols:
        df[col] = LabelEncoder().fit_transform(df[col])

    # One-hot encode nominal columns
    df = pd.get_dummies(df, columns=nominal_cols)

    print(f"Shape after encoding: {df.shape}")

    # Scale the numeric columns except for binary columns
    scale_cols = [col for col in num_cols if col not in binary_cols]
    print(f"Columns to scale: {scale_cols}")

    df[scale_cols] = StandardScaler().fit_transform(df[scale_cols])

    # Separate features and target
    X = df.drop(columns=[target_col])
    y = df[target_col]
    X = np.array(X)
    y = np.array(y)
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

# Adult Preprocess

In [273]:
def prepare_adult_data(file_path):
    """
    Function to preprocess the Adult dataset for classification.

    Parameters:
    - file_path: Path to the dataset file.

    Returns:
    - X_train, X_test, y_train, y_test: Preprocessed data split into training and testing sets.
    """
    # Define the column headers based on the dataset description
    columns = [
        'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
        'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
        'hours-per-week', 'native-country', 'income'
    ]

    # Load the dataset, handle missing values and whitespace issues
    adult_data = pd.read_csv(file_path, header=None, sep=',\s', na_values='?', engine='python', names=columns)

    # Display initial shape of the data
    print(f"Original data dimensions: {adult_data.shape}")

    # Count missing and duplicated values
    total_missing = adult_data.isnull().sum().sum()
    total_duplicates = adult_data.duplicated().sum()

    print(f"Total missing entries: {total_missing}")
    print(f"Total duplicate entries: {total_duplicates}")

    # Remove rows with missing target values and drop duplicates
    adult_data = adult_data.dropna(subset=['income']).drop_duplicates()

    # Clean the target column ('income') and map labels
    adult_data['income'] = adult_data['income'].str.strip().map({'>50K': 0, '<=50K': 1})

    # Fill missing numeric values with the column mean
    numeric_cols = adult_data.select_dtypes(include=['number']).columns
    adult_data[numeric_cols] = adult_data[numeric_cols].fillna(adult_data[numeric_cols].mean())

    # Fill missing categorical values with the mode
    categorical_cols = adult_data.select_dtypes(exclude=['number']).columns
    adult_data[categorical_cols] = adult_data[categorical_cols].fillna(adult_data[categorical_cols].mode().iloc[0])

    # Identify binary columns and nominal columns
    binary_cols = [col for col in adult_data.columns if adult_data[col].nunique() == 2]
    categorical_cols = adult_data.select_dtypes(include=['object']).columns.difference(binary_cols)

    print(f"Binary columns: {binary_cols}")
    print(f"Nominal columns: {list(categorical_cols)}")

    # Apply label encoding to binary columns
    le = LabelEncoder()
    for col in binary_cols:
        adult_data[col] = le.fit_transform(adult_data[col])

    # One-hot encoding for nominal categorical columns
    adult_data = pd.get_dummies(adult_data, columns=categorical_cols)

    # Scaling numeric columns except binary columns
    scale_columns = [col for col in numeric_cols if col not in binary_cols]
    print(f"Columns to be scaled: {scale_columns}")

    scaler = StandardScaler()
    adult_data[scale_columns] = scaler.fit_transform(adult_data[scale_columns])

    # Split dataset into features and target
    features = adult_data.drop(columns=['income'])
    target = adult_data['income']

    # Split the dataset into training and testing sets
    features = np.array(features)
    target = np.array(target)
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

# Credit Preprocess

In [274]:
def preprocess_credit_data():
    # Load dataset
    credit_data = pd.read_csv("creditcard.csv")

    # Display dataset shape
    print(f"Dataset dimensions: {credit_data.shape}")

    # Missing values check
    missing_values_count = credit_data.isnull().sum().sum()
    print(f"Total missing values: {missing_values_count}")

    # Duplicated values check
    duplicated_count = credit_data.duplicated().sum()
    print(f"Duplicated rows: {duplicated_count}")

    # Remove rows where 'Class' is missing
    credit_data.dropna(subset=['Class'], inplace=True)

    # Remove duplicate entries
    credit_data.drop_duplicates(inplace=True)

    # Fill missing values with column mean (only for numeric columns)
    num_columns = credit_data.select_dtypes(include='number').columns
    credit_data[num_columns] = credit_data[num_columns].fillna(credit_data[num_columns].mean())
    binary_cols = [column for column in credit_data.columns if credit_data[column].nunique() <= 2]
    nominal_cols = [column for column in credit_data.columns if credit_data[column].dtype == 'object' and column not in binary_cols]

    print(f"Binary columns: {binary_cols}")
    print(f"Nominal columns: {nominal_cols}")
    numeric_cols = credit_data.select_dtypes(include=['float64', 'int64']).columns
    scale_cols = [col for col in numeric_cols if col not in binary_cols]

    print(f"Columns to scale: {scale_cols}")
    scaler = StandardScaler()
    credit_data[scale_cols] = scaler.fit_transform(credit_data[scale_cols])

    # Convert boolean columns to integers
    bool_cols = credit_data.select_dtypes(include=['bool']).columns
    credit_data[bool_cols] = credit_data[bool_cols].astype(int)

    # Separate positive and negative 'Class' samples
    positive_samples = credit_data[credit_data['Class'] == 1]
    negative_samples = credit_data[credit_data['Class'] == 0].sample(n=20000, random_state=42)

    # Combine positive and sampled negative samples
    dataset_subset = pd.concat([positive_samples, negative_samples])

    print(f"Subset dimensions: {dataset_subset.shape}")
    X = dataset_subset.drop(columns=['Class'])
    y = dataset_subset['Class']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Further split training set into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

# Execute the preprocessing function


In [275]:
# from sklearn.preprocessing import LabelEncoder
# encoder = LabelEncoder()
# target = encoder.fit_transform(target)
# for i in features:
#     if features[i].dtypes == 'object':
#         features[i] = features[i].astype('category')
# features = pd.get_dummies(features)
# features


# Scaling

In [276]:
# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# def scaller(type):
#     lst = []
#     for i in features:
#         if features[i].dtypes != 'bool':
#             lst.append(i)
#     sc = StandardScaler()
#     if type == 'std':
#         sc = StandardScaler()
#     else:
#        sc = MinMaxScaler()
#     features[lst] = sc.fit_transform(features[lst])

# scaller('minmax')
# features

# Correlation Analysis

In [277]:
# features_df = pd.DataFrame(features, columns=features.columns)
# target_df = pd.DataFrame(target, columns=['Attrition'])

In [278]:
# # Correlation analysis of features with target
# target_series = target_df['Attrition']
# correlations = features.corrwith(target_series)
# correlations

In [279]:
# correlation_matrix = features_df.corr()
# correlation_matrix

In [280]:
# my_20_features = features[abs(correlations).sort_values(ascending=False).head(20).index]
# my_20_features

# 1D scatter plot

In [281]:

# import matplotlib.pyplot as plt
# import numpy as np
# class_0 = my_20_features.loc[target_df["Attrition"] == 0]
# class_1 = my_20_features.loc[target_df["Attrition"] == 1]

# # for column in my_20_features:
# #     plt.plot(class_0[column], np.zeros_like(class_0[column]), 'o', label='Class 0')
# #     plt.plot(class_1[column], np.zeros_like(class_1[column]), 'o', label='Class 1')
# #     plt.legend()
# #     plt.xlabel(column)
# #     plt.title("1D Scatter plot of " + column + " by Numeric classes")
# #     plt.show()



# Bagging

In [282]:


class MyBaggingClassifier:
    def __init__(self, base_estimator, n_estimators=9):
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.models = []

    def _bootstrap_sample(self, X, y):
        """Create a bootstrap sample of the dataset."""
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, size=n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        """Train the bagging ensemble of models."""
        self.models = []
        for _ in range(self.n_estimators):
            # Generate bootstrap sample
            X_sample, y_sample = self._bootstrap_sample(X, y)

            # Train a new model on the bootstrap sample
            model = self.base_estimator()
            model.fit(X_sample, y_sample)

            # Store the trained model
            self.models.append(model)

    def predict(self, X):
        """Make predictions using the ensemble of models."""
        # Collect predictions from each model
        predictions = np.array([model.predict(X) for model in self.models])

        # Majority voting: the most common prediction at each instance
        final_predictions = [Counter(predictions[:, i]).most_common(1)[0][0] for i in range(X.shape[0])]
        return np.array(final_predictions)

    def predict_proba(self, X):
        X = np.asarray(X, dtype=np.float64)
        """Get probability estimates from each model."""
        probas = np.array([model._sigmoid(np.dot(X, model.weights) + model.bias) for model in self.models])
        # Average probabilities across all models
        return np.mean(probas, axis=0)


**bold text**

# Logistic Regression

In [283]:
import numpy as np

class MyLogisticRegression:
    def __init__(self, learning_rate=0.3, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None

    def _sigmoid(self, z):
        """Apply the sigmoid function."""
        return 1 / (1 + np.exp(-z))

    def _initialize_parameters(self, n_features):
        """Initialize weights and bias to zero."""
        self.weights = np.zeros(n_features)
        self.bias = 0

    def _compute_loss(self, y_true, y_predicted):
        """Compute the binary cross-entropy loss."""
        n_samples = len(y_true)
        # Loss = - 1/n * Σ [y * log(p) + (1 - y) * log(1 - p)]
        loss = (-1 / n_samples) * np.sum(y_true * np.log(y_predicted) + (1 - y_true) * np.log(1 - y_predicted))
        return loss

    def _update_weights(self, X, y_true, y_predicted):
        """Perform a single update of gradient descent."""
        n_samples = X.shape[0]
        # Compute the gradients
        dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y_true))
        db = (1 / n_samples) * np.sum(y_predicted - y_true)

        # Update weights and bias
        self.weights -= self.learning_rate * dw
        self.bias -= self.learning_rate * db

    def fit(self, X, y):
        """Fit the logistic regression model to the data."""
        X = np.asarray(X, dtype=np.float64)
        y = np.asarray(y, dtype=np.float64)
        n_samples, n_features = X.shape

        # Initialize weights and bias
        self._initialize_parameters(n_features)

        # Gradient descent
        for i in range(self.n_iterations):
            # Linear model
            linear_model = np.dot(X, self.weights) + self.bias
            # Apply sigmoid function
            y_predicted = self._sigmoid(linear_model)

            # Compute the loss (optional, but useful for monitoring)
            loss = self._compute_loss(y, y_predicted)
            # if i % 100 == 0:
            #     print(f"Iteration {i}, Loss: {loss}")

            # Update weights and bias using gradient descent
            self._update_weights(X, y, y_predicted)

    def predict(self, X):
        """Predict class labels for samples in X."""
        X = np.asarray(X, dtype=np.float64)
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        y_predicted_labels = [1 if i > 0.5 else 0 for i in y_predicted]
        return np.array(y_predicted_labels)


# Example usage:
# if __name__ == "__main__":
#     # Sample data (features and labels)
#     X = np.array([[1, 2], [2, 3], [3, 4], [4, 5], [5, 6]])
#     y = np.array([0, 0, 0, 1, 1])

#     # Create and train the model
#     model = MyLogisticRegression(learning_rate=0.01, n_iterations=1000)
#     model.fit(X, y)

#     # Make predictions
#     predictions = model.predict(X)
#     print("Predictions:", predictions)


# Stacking

In [284]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, average_precision_score

class MyStackingClassifier:
    def __init__(self, base_estimators, meta_estimator):
        self.base_estimators = base_estimators
        self.meta_estimator = meta_estimator

    def fit(self, X_train, y_train, X_val, y_val):
        """Fit the base models and then the meta-learner."""
        meta_features = []

        # Train each base estimator on the training data
        for estimator in self.base_estimators:
            estimator.fit(X_train, y_train)
            # Generate meta-features by making predictions on the validation set
            meta_features.append(estimator.predict_proba(X_val))

        # Stack the meta-features
        meta_features = np.column_stack(meta_features)

        # Train the meta-learner on the meta-features using the validation labels
        self.meta_estimator.fit(meta_features, y_val)

    def predict(self, X_test):
        """Make predictions on the test set using the base models and the meta-learner."""
        # Get meta-features for the test set by making predictions using the base models
        meta_features = []
        for estimator in self.base_estimators:
            meta_features.append(estimator.predict_proba(X_test))

        # Stack the meta-features
        meta_features = np.column_stack(meta_features)

        # Make final predictions using the meta-learner
        return self.meta_estimator.predict(meta_features)

    def evaluate(self, X_test, y_test):
        """Evaluate the model and calculate the required metrics."""
        # Predict using the test data
        y_pred = self.predict(X_test)

        # Compute the metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()  # true negatives, false positives, false negatives, true positives
        specificity = tn / (tn + fp)
        auc = roc_auc_score(y_test, y_pred)
        aupr = average_precision_score(y_test, y_pred)

        # Print the metrics
        print("Accuracy:", accuracy)
        print("Precision:", precision)
        print("Recall:", recall)
        print("F1 Score:", f1)
        print("Specificity:", specificity)
        print("AUC ROC:", auc)
        print("AUPR:", aupr)

        # Return the metrics as a dictionary (optional)
        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'specificity': specificity,
            'auc_roc': auc,
            'aupr': aupr
        }

# Example usage
# Create an instance of MyStackingClassifier (make sure you define your


# Violine Plot

In [285]:
# Function to calculate multiple performance metrics
def evaluate_model_performance(y_true, y_pred):
    """Calculate accuracy, precision, recall, F1, specificity, AUC-ROC, and AUC-PR."""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    specificity = tn / (tn + fp)
    auc = roc_auc_score(y_true, y_pred)
    aupr = average_precision_score(y_true, y_pred)

    return accuracy, precision, recall, f1, specificity, auc, aupr

# Function to collect metrics from all 9 base learners
def collect_metrics_from_bagging(bagging_classifier, X_test, y_test):
    metrics_list = []
    for model in bagging_classifier.models:
        y_pred = model.predict(X_test)
        accuracy, precision, recall, f1, specificity, auc, aupr = evaluate_model_performance(y_test, y_pred)
        metrics_list.append([accuracy, f1, recall, precision, specificity, auc, aupr])
    return metrics_list

# Function to plot the violin plot for all metrics across the 9 learners
def plot_violin_for_metrics(metrics_list):
    # Convert the metrics list to a pandas DataFrame
    metrics_df = pd.DataFrame(metrics_list, columns=["Accuracy", "F1", "Sensitivity", "Precision", "Specificity", "AUROC", "AUPR"])

    # Plot using seaborn's violinplot
    plt.figure(figsize=(12, 8))
    sns.violinplot(data=metrics_df, inner="point")
    plt.title("Performance Metrics of Bagged Logistic Regression Models")
    plt.xlabel("Metric")
    plt.ylabel("Metric Score")
    plt.xticks(ticks=np.arange(len(metrics_df.columns)), labels=metrics_df.columns)  # Ensure proper labeling
    plt.show()

In [287]:
# Import necessary libraries
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Assume X and y are your input matrices
# X --> (number of rows, number of columns), already scaled
# y --> binary target class (0 or 1)
# Dummy example data (replace these with your actual data)
# X = np.random.rand(100, 5) # Example feature matrix with 100 rows and 5 columns
# y = np.random.randint(0, 2, 100) # Example binary target vector
# Step 1: Split the data into training and testing sets
# X = np.array(my_20_features)
# y = np.array(target)
X_train, X_test, y_train, y_test = preprocess_credit_data()
X_trainBag, X_val, y_trainBag, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
# Create bagging classifiers with MyLogisticRegression as the base model
bagging_classifiers = [MyBaggingClassifier(base_estimator=MyLogisticRegression, n_estimators=9) for _ in range(3)]

# Use MyBaggingClassifier exactly as named in your code
bagging_classifier = MyBaggingClassifier(base_estimator=MyLogisticRegression, n_estimators=9)

# Train the bagging classifier
bagging_classifier.fit(X_train, y_train)

# Collect the performance metrics from all the 9 base learners
metrics_list = collect_metrics_from_bagging(bagging_classifier, X_test, y_test)

# Plot the violin plot for the metrics collected from the bagging classifier
plot_violin_for_metrics(metrics_list)

# Meta-learner
meta_learner = MyLogisticRegression()

# Create stacking classifier
stacking_classifier = MyStackingClassifier(base_estimators=bagging_classifiers, meta_estimator=meta_learner)

# Train the stacking classifier using the validation set as meta-training data
stacking_classifier.fit(X_trainBag, y_trainBag, X_val, y_val)
stacking_classifier.evaluate(X_test, y_test)
# Make predictions on the test set
y_pred = stacking_classifier.predict(X_test)
# print("Test Predictions:", y_pred)

# # Compare predictions with true values
# print("True Test Labels:", y_test)

# bagging_accuracies = []
# for classifier in bagging_classifiers:
#     accuracy = accuracy_score(y_test, classifier.predict(X_test))
#     bagging_accuracies.append(accuracy)

# # Plot the violin plot for the bagging accuracies
# plot_violin(bagging_accuracies)

# bagging_classifier = MyBaggingClassifier(base_estimator=MyLogisticRegression, n_estimators=9)
# bagging_classifier.fit(X_trainBag, y_trainBag)
# predictions = bagging_classifier.predict(X_val)
# print("Validation Predictions:", predictions)

# # Compare predictions with true values
# print("True Validation Labels:", y_val)

# y_pred = bagging_classifier.predict(X_val)
accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy of Bagging classifier: {accuracy:.2f}"


# # Step 2: Initialize the Logistic Regression classifier
# clf =  MyLogisticRegression(learning_rate=0.01, n_iterations=1000)
# # Step 3: Train the classifier on the training data
# clf.fit(X_train, y_train)
# # Step 4: Make predictions on the test set
# y_pred = clf.predict(X_test)
# # Step 5: Evaluate the classifier's performance
# accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of Logistic Regression classifier: {accuracy:.2f}")

Dataset dimensions: (284807, 31)
Total missing values: 0
Duplicated rows: 1081
Binary columns: ['Class']
Nominal columns: []
Columns to scale: ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']
Subset dimensions: (20473, 31)


KeyError: "None of [Index([ 3794, 13421, 10724,  7932,  5306, 15390,  9562, 10470, 14596, 12396,\n       ...\n        1212, 11220, 10501, 10067,  4435, 11311,  2353,  6995, 14091, 11996],\n      dtype='int64', length=16378)] are in the [columns]"

# Table generation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, confusion_matrix

# Function to calculate multiple performance metrics
def evaluate_model_performance(y_true, y_pred):
    """Calculate accuracy, precision, recall, F1, specificity, AUC-ROC, and AUC-PR."""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    specificity = tn / (tn + fp)
    auc = roc_auc_score(y_true, y_pred)
    aupr = average_precision_score(y_true, y_pred)

    return [accuracy, precision, recall, f1, specificity, auc, aupr]

# Function to collect metrics from all base learners in the bagging model
def collect_metrics_from_bagging(bagging_classifier, X_test, y_test):
    metrics_list = []
    for model in bagging_classifier.models:
        y_pred = model.predict(X_test)
        metrics = evaluate_model_performance(y_test, y_pred)
        metrics_list.append(metrics)
    return metrics_list

# Function to compute average and std deviation for bagged learners
def summarize_bagging_performance(metrics_list):
    metrics_array = np.array(metrics_list)
    means = metrics_array.mean(axis=0)
    stds = metrics_array.std(axis=0)
    return means, stds

# Function to create the performance table
def create_performance_table(models_metrics):
    # Create a pandas DataFrame to display results
    metrics_df = pd.DataFrame(models_metrics, columns=["Model", "Accuracy", "Sensitivity", "Specificity", "Precision", "F1-score", "AUROC", "AUPR"])
    return metrics_df

def custom_majority_voting(base_estimators, X_test):

    # Collect predictions from each base estimator
    predictions = np.array([estimator.predict(X_test) for estimator in base_estimators])

    # Apply majority voting by taking the most common prediction (mode) along axis 0
    majority_votes = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)

    return majority_votes

# Collect performance metrics for different models
models_metrics = []

# Bagging LR learners (Assuming `bagging_classifier` is trained and has 9 learners)
bagging_classifier = MyBaggingClassifier(base_estimator=MyLogisticRegression, n_estimators=9)
bagging_classifier.fit(X_train, y_train)  # Don't forget to fit the bagging classifier
bagging_metrics_list = collect_metrics_from_bagging(bagging_classifier, X_test, y_test)
bagging_means, bagging_stds = summarize_bagging_performance(bagging_metrics_list)

# Append LR bagging metrics (mean ± std)
models_metrics.append([
    f"LR (Bagging)",
    f"{bagging_means[0]:.4f} ± {bagging_stds[0]:.4f}",  # Accuracy
    f"{bagging_means[2]:.4f} ± {bagging_stds[2]:.4f}",  # Sensitivity
    f"{bagging_means[4]:.4f} ± {bagging_stds[4]:.4f}",  # Specificity
    f"{bagging_means[1]:.4f} ± {bagging_stds[1]:.4f}",  # Precision
    f"{bagging_means[3]:.4f} ± {bagging_stds[3]:.4f}",  # F1-score
    f"{bagging_means[5]:.4f} ± {bagging_stds[5]:.4f}",  # AUROC
    f"{bagging_means[6]:.4f} ± {bagging_stds[6]:.4f}"   # AUPR
])

# Custom Voting Ensemble (Using the custom majority voting function)
models = bagging_classifier.models
y_pred_voting = custom_majority_voting(models, X_test)
voting_metrics = evaluate_model_performance(y_test, y_pred_voting)
models_metrics.append(["Voting Ensemble"] + [f"{metric:.4f}" for metric in voting_metrics])

# Stacking ensemble (Assuming `stacking_classifier` is trained)
stacking_metrics = stacking_classifier.evaluate(X_test, y_test)
models_metrics.append([
    "Stacking Ensemble",
    f"{stacking_metrics['accuracy']:.4f}",
    f"{stacking_metrics['recall']:.4f}",   # Sensitivity (Recall)
    f"{stacking_metrics['specificity']:.4f}",
    f"{stacking_metrics['precision']:.4f}",
    f"{stacking_metrics['f1_score']:.4f}",
    f"{stacking_metrics['auc_roc']:.4f}",
    f"{stacking_metrics['aupr']:.4f}"
])

# Create the performance table
performance_table = create_performance_table(models_metrics)


performance_table
